<a href="https://colab.research.google.com/github/Bast-aleja/Marketing_1/blob/main/Trabajo_peliculasreal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.PROBLEMA DE NEGOCIO 
1.1 Descripción del problema de negocio

Esta plataforma online tiene inconvenientes en cuanto a la fidelización de clientes debido a que las demás plataformas brindan mejores experiencias dándole gusto al usuario y esta plataforma aún no lo ha empleado una estrategia competitiva y por ende el tema de tener nuevos clientes es más complejo.

1.2 Decripción del problema analítico

1.	Analizar el comportamiento y los hábitos de sus consumidores en la plataforma online
2.	Predecir contenido afín (películas) a los gustos que muestra el consumo que ha tenido el usuario.

1.3 Diseño de la solución 

En esta parte se realiza un diagrama de flujo con el paso a paso de la solución propuesta para este problema del área de marketing, se comienza con el análisis de consumo de los usuarios y finalmente se propone un plan de conteiido recomendado

**CODIGO**

Importación de paquetes

In [1]:
#Importación de paquetes
import pandas as pd
import numpy as np
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import requests


Lectura de datos

In [2]:
#Conexión con colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
conn = sql.connect('db_movies')
cur= conn.cursor()

In [5]:
#Leer cantidad de bases que tiene el archivo
tables = pd.read_sql("SELECT name FROM sqlite_master where type = 'table'", conn)
tables

,name
0,ratings
1,movies


Lectura base de datos: movies

In [ ]:
movies = pd.read_sql("SELECT * FROM movies", conn)
movies

Depuración BD movies

In [7]:
#Duplicados
movies.duplicated().sum()

0

In [ ]:
#Analizar dimensión, tipos y nulos
print(movies.shape)
print('-------------------------------------------------')
print(movies.isnull().sum())
print('-------------------------------------------------')
print(movies.dtypes)
print('-------------------------------------------------')

In [ ]:
#separa columna generos 


from mlxtend.preprocessing import TransactionEncoder

conn= sql.connect('db_movies')

movies = pd.read_sql('SELECT * FROM movies', conn)
genres=movies['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genress = pd.DataFrame(genres, columns = te.columns_)
genress


Unión categoria genres a la base movies

In [11]:
movies.drop(columns=['genres'], inplace =True)


In [12]:
movies = pd.concat ([movies, genress], axis = 1)

In [ ]:
#relacionar los verdaderos con 1 y los falsos con 0
dic_genres = {
    False:0,
    True:1
}
movies[genress.columns] = movies[genress.columns].replace(dic_genres )
movies

In [ ]:
#Se arregla los tipos de datos de las columnas
movies=movies.convert_dtypes()
movies.dtypes

In [ ]:
#Cambiar el nombre de las columnas a minusculas
movies.columns = movies.columns.str.lower()
movies.columns

In [17]:
#Separar el año
movies['y'] = movies['title'].str.split(' ')
movies['year'] = movies['y'].apply(lambda x: x[-1])
movies = movies.drop(['y'], axis = 1)
movies['year'] = movies['year'].str.strip(')')
movies['year'] = movies['year'].str.strip('(')


In [18]:
#Mirar si existen datos nulos
movies['year'].isnull().sum()


0

In [19]:
#Mirar si existen años mal escritos
movies['year'].unique()

array(['1995', '1994', '1996', '1976', '1992', '1967', '1993', '1964',
       '1977', '1965', '1982', '1990', '1991', '1989', '1937', '1940',
       '1969', '1981', '1973', '1970', '1955', '1959', '1968', '1988',
       '1997', '1972', '1943', '1952', '1951', '1957', '1961', '1958',
       '1954', '1934', '1944', '1960', '1963', '1942', '1941', '1953',
       '1939', '1950', '1946', '1945', '1938', '1947', '1935', '1936',
       '1956', '1949', '1932', '1975', '1974', '1971', '1979', '1987',
       '1986', '1980', '1978', '1985', '1966', '1962', '1983', '1984',
       '1948', '1933', '1931', '1922', '1998', '1929', '1930', '1927',
       '1928', '1999', '2000', '1926', '1919', '1921', '1925', '1923',
       '2001', '2002', '2003', '1920', '1915', '1924', '2004', '1916',
       '1917', '', '2005', '2006', '1902', '5', '1903', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '1908', 'One', 'Road', 'Watson', 'Animals',
       'P

In [ ]:
#Cuantos datos representan 
movies.query("year=='' or year=='5' or year== 'One' or year=='Road'or year=='Watson' or year=='Animals' or year=='Paterson'or year=='Moonlight' or year=='Moonlight' or year=='OA' or year=='Cosmos' or year=='Baby' or year=='20006-2007' or year=='2' or year=='Mirror'")

In [32]:
#Se reemplazan los 23 datos en years escritos incorrectamente
lista= ('Cosmos', 'OA', 'One', 'Paterson', 'Moonlight', 'Watson', '2', 'Animals', 'Baby', 'Mirror','Road','','5')


Se encuentra que existen muy pocos datos (23 de 9742) así que se puede optar por borrarlos o reemplazar sus valores. Se tomara aquel año que más se repite y serán reemplazados por ese año ya que los nombres escritos no dan indicios de una fecha excepto por 2006-2007 que se tomara el 2007

In [33]:
#Busco el dato que más se repite para que reemplace los valores

movies['year'].describe()

count     9742
unique     120
top       2002
freq       311
Name: year, dtype: object

In [39]:
#Reemplazar los datos que no son fechas

movies['year'] = movies['year'].replace(('2006–2007','2006'))

for i in lista:
  movies['year'] = movies['year'].replace((i, '2002'))



In [48]:
#poner el año en el tipo de dato adecuado
movies['year'] = movies['year'].astype('int64')

In [50]:
#Homologación de categorias 
listaf= ['title',
        '(no genres listed)', 'action', 'adventure',
       'animation', 'children', 'comedy', 'crime', 'documentary', 'drama',
       'fantasy', 'film-noir', 'horror', 'imax', 'musical', 'mystery',
       'romance', 'sci-fi', 'thriller', 'war', 'western', 'year']

In [ ]:
for i in listaf:
  print(movies[i].value_counts())
  print('-----------------------------------------------')

Lectura base de datos: raitings

In [53]:
#Lectura DB ratingS
ratings=pd.read_sql("""select *,datetime(timestamp , 'unixepoch') as date from ratings""", conn)
ratings

,userId,movieId,rating,timestamp,date
0,1,1,4.0,964982703,2000-07-30 18:45:03
1,1,3,4.0,964981247,2000-07-30 18:20:47
2,1,6,4.0,964982224,2000-07-30 18:37:04
3,1,47,5.0,964983815,2000-07-30 19:03:35
4,1,50,5.0,964982931,2000-07-30 18:48:51
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,2017-05-03 21:53:22
100832,610,168248,5.0,1493850091,2017-05-03 22:21:31
100833,610,168250,5.0,1494273047,2017-05-08 19:50:47
100834,610,168252,5.0,1493846352,2017-05-03 21:19:12


**Depuración DB ratings**

In [54]:
#información general
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
 4   date       100836 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 3.8+ MB


In [55]:
#Duplicados
ratings.duplicated().sum()

0

In [56]:
#Analizar dimensión, tipos y nulos
print(ratings.shape)
print('-------------------------------------------------')
print(ratings.isnull().sum())
print('-------------------------------------------------')
print(ratings.dtypes)
print('-------------------------------------------------')

(100836, 5)
-------------------------------------------------
userId       0
movieId      0
rating       0
timestamp    0
date         0
dtype: int64
-------------------------------------------------
userId         int64
movieId        int64
rating       float64
timestamp      int64
date          object
dtype: object
-------------------------------------------------


In [57]:
#Se arregla los tipos de datos de las columnas
ratings=ratings.convert_dtypes()
ratings.dtypes

userId         Int64
movieId        Int64
rating       Float64
timestamp      Int64
date          string
dtype: object

In [58]:
#Se le quita la hora a la fecha
ratings['date']=ratings['date'].apply(lambda x:str(x)[0:10])

In [68]:
#Se le quita la columna timestap ya que al convertirla a date ya no es necesaria
ratings=ratings.drop(['timestamp'], axis=1)

In [61]:
#Se arregla el tipo de dato de date a fecha

ratings['date']= pd.to_datetime(ratings['date'], errors= 'coerce', format='%Y-%m-%d')
ratings.dtypes

userId                Int64
movieId               Int64
rating              Float64
timestamp             Int64
date         datetime64[ns]
dtype: object

In [62]:
#Cambiar el nombre de las columnas a minusculas
ratings.columns = ratings.columns.str.lower()
ratings.columns

Index(['userid', 'movieid', 'rating', 'timestamp', 'date'], dtype='object')

In [63]:
#Homologación de categorias 
#timestap tiene la misma fecha para todos (no sirve?-eliminar?)
listaf= ['rating', 'date']

In [64]:
for i in listaf:
  print(ratings[i].value_counts())
  print('-----------------------------------------------')

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: Int64
-----------------------------------------------
2017-06-26    1014
2007-05-14     878
2017-05-03     866
2000-08-08     709
2015-06-28     606
              ... 
2010-01-09       1
2005-03-27       1
2005-08-14       1
2003-10-29       1
2018-03-08       1
Name: date, Length: 4110, dtype: int64
-----------------------------------------------


In [70]:
#Se guardan las bases de datos finales
movies = movies.copy(deep = True)
ratings =ratings.copy(deep= True)

Preguntas movies

In [ ]:
#movies['title']=movies.title.str.replace(' (\(\d\d\d\d\))', '')
movies['year'].value_counts()

In [ ]:
movies['title']=movies.title.str.replace(' (\(\d\d\d\d\))', '')

In [ ]:
#se quita el año a los nombres
movies['title'].str.split('\s+\(' , expand=True)[0]

In [ ]:
#número de peliculas lanzadas por año
movies.groupby(['year'])[['title']].count().reset_index()

In [ ]:
#Crear una columna llamada año -------> ver para qué necesitamos esta fecha
ratings['año'] = ratings['timestamp'].dt.year
ratings

,userId,movieId,rating,timestamp,date,año
0,1,1,4.0,1970-01-01 00:00:00.964982703,2000-07-30 18:45:03,1970
1,1,3,4.0,1970-01-01 00:00:00.964981247,2000-07-30 18:20:47,1970
2,1,6,4.0,1970-01-01 00:00:00.964982224,2000-07-30 18:37:04,1970
3,1,47,5.0,1970-01-01 00:00:00.964983815,2000-07-30 19:03:35,1970
4,1,50,5.0,1970-01-01 00:00:00.964982931,2000-07-30 18:48:51,1970
...,...,...,...,...,...,...
100831,610,166534,4.0,1970-01-01 00:00:01.493848402,2017-05-03 21:53:22,1970
100832,610,168248,5.0,1970-01-01 00:00:01.493850091,2017-05-03 22:21:31,1970
100833,610,168250,5.0,1970-01-01 00:00:01.494273047,2017-05-08 19:50:47,1970
100834,610,168252,5.0,1970-01-01 00:00:01.493846352,2017-05-03 21:19:12,1970


In [ ]:
# cantidad de peliculas por genero
#df = pd.read_sql("""SELECT * FROM movies""", conn)
#df = df.sort_value('sum', ascending = False)
#df

Graficas ratings

In [ ]:
#Se mira que peliculas tuvieron una calificación de 1
cr=pd.read_sql(""" select 
                          movieid, rating,
                          count(*) as cnt
                          from ratings
                          where rating = 1.0
                          group by movieid
                          order by cnt desc
                          """, conn)
cr


,movieId,rating,cnt
0,231,1.0,12
1,2710,1.0,11
2,19,1.0,10
3,344,1.0,10
4,367,1.0,10
...,...,...,...
1721,187593,1.0,1
1722,189547,1.0,1
1723,190213,1.0,1
1724,190219,1.0,1


In [ ]:
movies.drop(columns=['genres'], inplace =True)

In [ ]:
##### Descripción base de ratings
cr=pd.read_sql(""" select 
                          "rating" as ratings, 
                          count(*) as conteo 
                          from ratings
                          group by "rating"
                          """, conn)
###Nombres de columnas con numeros o guiones se deben poner en doble comilla para que se reconozcan

In [ ]:
data  = go.Bar( x=cr.ratings,y=cr.conteo, text=cr.conteo, textposition="outside")
Layout=go.Layout(title="Conteo de calificaciones",xaxis={'title':'Calificación'},yaxis={'title':'Total'})
go.Figure(data,Layout)

In [ ]:
#CANTIDAD DE PERSONAS QUE MIRAN UNA PELICULA-POCOS USUARIOS HAN VISTO MAS DE 50
pc=pd.read_sql(""" select 
                          "useaId" as userid, 
                          count(*) as contar 
                          from ratings
                          group by "movieId"
                          order by contar desc
                          """, conn)
pc

,userid,contar
0,useaId,329
1,useaId,317
2,useaId,307
3,useaId,279
4,useaId,278
...,...,...
9719,useaId,1
9720,useaId,1
9721,useaId,1
9722,useaId,1


In [ ]:
data  = go.Scatter(x = pc.index, y= pc.contar)
Layout= go.Layout(title="PELICULAS VISTAS POR USUARIO",xaxis={'title':'Userios'}, yaxis={'title':'Peliculas vistas'})
go.Figure(data, Layout)  

Unir ambas tablas 

In [ ]:
movies

,movieid,title,(no genres listed),action,adventure,animation,children,comedy,crime,documentary,...,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western,year
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1995
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1995
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2017
9738,193583,No Game No Life: Zero (2017),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2017
9739,193585,Flint (2017),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017
9740,193587,Bungo Stray Dogs: Dead Apple (2018),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2018


In [ ]:
#execution = "SELECT * FROM movies INNER JOIN ratings on ratings.movieid = movies.movieid"
#movies_ratings = pd.read_sql(execution, conn)
#movies_ratings

In [ ]:
union = ratings.merge(movies, on= 'movieid', how='inner')

In [ ]:
union

,userid,movieid,rating,timestamp,date,año,title,(no genres listed),action,adventure,...,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western,year
0,1,1,4.0,1970-01-01 00:00:00.964982703,2000-07-30 18:45:03,1970,Toy Story,0,0,1,...,0,0,0,0,0,0,0,0,0,1995
1,5,1,4.0,1970-01-01 00:00:00.847434962,1996-11-08 06:36:02,1970,Toy Story,0,0,1,...,0,0,0,0,0,0,0,0,0,1995
2,7,1,4.5,1970-01-01 00:00:01.106635946,2005-01-25 06:52:26,1970,Toy Story,0,0,1,...,0,0,0,0,0,0,0,0,0,1995
3,15,1,2.5,1970-01-01 00:00:01.510577970,2017-11-13 12:59:30,1970,Toy Story,0,0,1,...,0,0,0,0,0,0,0,0,0,1995
4,17,1,4.5,1970-01-01 00:00:01.305696483,2011-05-18 05:28:03,1970,Toy Story,0,0,1,...,0,0,0,0,0,0,0,0,0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,160341,2.5,1970-01-01 00:00:01.479545749,2016-11-19 08:55:49,1970,Bloodmoon,0,1,0,...,0,0,0,0,0,0,1,0,0,1997
100832,610,160527,4.5,1970-01-01 00:00:01.479544998,2016-11-19 08:43:18,1970,Sympathy for the Underdog,0,1,0,...,0,0,0,0,0,0,0,0,0,1971
100833,610,160836,3.0,1970-01-01 00:00:01.493844794,2017-05-03 20:53:14,1970,Hazard,0,1,0,...,0,0,0,0,0,0,1,0,0,2005
100834,610,163937,3.5,1970-01-01 00:00:01.493848789,2017-05-03 21:59:49,1970,Blair Witch,0,0,0,...,1,0,0,0,0,0,1,0,0,2016


In [ ]:
print(union.isnull().sum())
union.shape

userid                0
movieid               0
rating                0
timestamp             0
date                  0
año                   0
title                 0
(no genres listed)    0
action                0
adventure             0
animation             0
children              0
comedy                0
crime                 0
documentary           0
drama                 0
fantasy               0
film-noir             0
horror                0
imax                  0
musical               0
mystery               0
romance               0
sci-fi                0
thriller              0
war                   0
western               0
year                  0
dtype: int64


(100836, 28)

In [ ]:
#movies_ratings.info()

In [ ]:
#movies_ratings.duplicated().sum()

Modelos

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# paquetes
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD
from surprise.model_selection import train_test_split

In [ ]:
############################################################################
#####1. Sistema de recomendación filtro colaborativo basado en usuario #####
############################################################################


union.to_sql('ratings', conn)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
###### leer datos desde tabla de pandas
conn = sql.connect('ratings')
cur=conn.cursor()


In [ ]:
ratings = pd.read_sql('select * from ratings', conn)

In [ ]:
###### leer datos desde tabla de pandas
reader = Reader(rating_scale=(0, 10))

In [ ]:
###las columnas deben estar en orden estándar: user item rating
data   = Dataset.load_from_df(ratings[['userid','movieid','rating']], reader)

In [ ]:
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()] 
results = {}

In [ ]:
for model in models:
 
    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)  
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.669232,0.875510,0.254661,2.410937
knns.KNNWithZScore,0.681059,0.897902,0.230537,2.143897
knns.KNNWithMeans,0.686467,0.898745,0.145181,1.961734
knns.KNNBasic,0.726509,0.947706,0.178297,2.009584


se escoge knns.KNNWithMeans porque tiene menor tiempo computacional

In [ ]:
param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5], \
                                'user_based': [False, True]}
             }

In [ ]:
#MAE
gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['mae'], \
                                      cv=2, n_jobs=2)

gridsearchKNNWithMeans.fit(data)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
gridsearchKNNWithMeans.best_params["mae"]


{'sim_options': {'min_support': 5, 'name': 'msd', 'user_based': True}}

In [ ]:
gridsearchKNNWithMeans.best_score["mae"]

0.7079871290841573

In [ ]:
#RMSE
gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=2)
                                    
gridsearchKNNWithMeans.fit(data)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
gridsearchKNNWithMeans.best_params["rmse"]


{'sim_options': {'min_support': 5, 'name': 'msd', 'user_based': True}}

In [ ]:
gridsearchKNNWithMeans.best_score["rmse"]

0.924887997760371

**Predicciones**

In [ ]:
trainset = data.build_full_trainset()

In [ ]:
sim_options       = {'name':'msd','min_support':5,'user_based':True}
model = KNNWithMeans(sim_options=sim_options)
model=model.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
predset = trainset.build_anti_testset() 
predictions = model.test(predset) ### función muy pesada
predictions_df = pd.DataFrame(predictions)
predictions

[Prediction(uid=1, iid=318, r_ui=3.501556983616962, est=4.900272677469961, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=1704, r_ui=3.501556983616962, est=4.762763346032554, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=6874, r_ui=3.501556983616962, est=4.689041558103872, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=8798, r_ui=3.501556983616962, est=4.543513120959999, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=46970, r_ui=3.501556983616962, est=4.138355099192787, details={'actual_k': 27, 'was_impossible': False}),
 Prediction(uid=1, iid=48516, r_ui=3.501556983616962, est=4.766093136958814, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=58559, r_ui=3.501556983616962, est=4.718815313132854, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=60756, r_ui=3.501556983616962, est=4.323852480105616, details={'actual_k': 27, 

In [ ]:
def recomendaciones(userid,n_recomend=10):
    
    predictions_userID = predictions_df[predictions_df['uid'] ==userid].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendados = predictions_userID[['iid','est']]
    recomendados.to_sql('reco',conn,if_exists="replace")
    
    recomendados=pd.read_sql('''select a.*, b.movieid 
                             from reco a left join movieid b
                             on a.iid=b.isbn ''', conn)

    return(recomendados)

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
predictions_df['uid'].unique()[:20] 

array([ 1,  5,  7, 15, 17, 18, 19, 21, 27, 31, 32, 33, 40, 43, 44, 45, 46,
       50, 54, 57])

In [ ]:
us1=recomendaciones(userid=179733,n_recomend=20)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 45))



DatabaseError: ignored